## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_Name,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Yar-Sale,RU,66.8333,70.8333,15.04,86,100,8.77,overcast clouds
1,1,Lebu,CL,-37.6167,-73.6500,52.25,91,21,7.14,few clouds
2,2,Namtsy,RU,62.7161,129.6658,33.21,80,100,13.15,overcast clouds
3,3,Kashary,RU,49.0400,41.0056,53.94,68,100,9.33,overcast clouds
4,4,Progreso,MX,21.2833,-89.6667,82.11,80,0,15.12,clear sky


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_Name,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
4,4,Progreso,MX,21.2833,-89.6667,82.11,80,0,15.12,clear sky
5,5,Biak,ID,-0.9131,122.8766,83.35,66,100,1.86,overcast clouds
11,11,Boa Vista,BR,2.8197,-60.6733,82.38,65,40,10.36,scattered clouds
14,14,Socorro,US,31.6546,-106.3033,82.18,5,20,6.91,few clouds
16,16,Atuona,PF,-9.8000,-139.0333,79.74,79,12,18.30,few clouds
17,17,Butaritari,KI,3.0707,172.7902,81.59,73,30,12.71,scattered clouds
18,18,Noumea,NC,-22.2763,166.4572,82.99,99,36,8.25,scattered clouds
22,22,Rikitea,PF,-23.1203,-134.9692,78.48,63,2,10.27,clear sky
23,23,Malakal,SS,9.5334,31.6605,84.07,46,9,4.34,clear sky
25,25,Hithadhoo,MV,-0.6000,73.0833,82.99,75,29,10.40,scattered clouds


In [8]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_Name      185
City           185
Country        185
Lat            185
Lng            185
Max Temp       185
Humidity       185
Cloudiness     185
Wind Speed     185
Description    185
dtype: int64

In [13]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_Name      185
City           185
Country        185
Lat            185
Lng            185
Max Temp       185
Humidity       185
Cloudiness     185
Wind Speed     185
Description    185
dtype: int64

In [15]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
4,Progreso,MX,82.11,clear sky,21.2833,-89.6667,
5,Biak,ID,83.35,overcast clouds,-0.9131,122.8766,
11,Boa Vista,BR,82.38,scattered clouds,2.8197,-60.6733,
14,Socorro,US,82.18,few clouds,31.6546,-106.3033,
16,Atuona,PF,79.74,few clouds,-9.8000,-139.0333,
17,Butaritari,KI,81.59,scattered clouds,3.0707,172.7902,
18,Noumea,NC,82.99,scattered clouds,-22.2763,166.4572,
22,Rikitea,PF,78.48,clear sky,-23.1203,-134.9692,
23,Malakal,SS,84.07,clear sky,9.5334,31.6605,
25,Hithadhoo,MV,82.99,scattered clouds,-0.6000,73.0833,


In [17]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json() 
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
try:
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
except (IndexError):
    print("Hotel not found... skipping.")
    
hotel_df.head(10)  
        

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
4,Progreso,MX,82.11,clear sky,21.2833,-89.6667,
5,Biak,ID,83.35,overcast clouds,-0.9131,122.8766,
11,Boa Vista,BR,82.38,scattered clouds,2.8197,-60.6733,
14,Socorro,US,82.18,few clouds,31.6546,-106.3033,
16,Atuona,PF,79.74,few clouds,-9.8000,-139.0333,
17,Butaritari,KI,81.59,scattered clouds,3.0707,172.7902,
18,Noumea,NC,82.99,scattered clouds,-22.2763,166.4572,
22,Rikitea,PF,78.48,clear sky,-23.1203,-134.9692,
23,Malakal,SS,84.07,clear sky,9.5334,31.6605,
25,Hithadhoo,MV,82.99,scattered clouds,-0.6000,73.0833,


In [18]:
# 7. Drop the rows where there is no Hotel Name.
clean_df = hotel_df.dropna()
clean_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
4,Progreso,MX,82.11,clear sky,21.2833,-89.6667,
5,Biak,ID,83.35,overcast clouds,-0.9131,122.8766,
11,Boa Vista,BR,82.38,scattered clouds,2.8197,-60.6733,
14,Socorro,US,82.18,few clouds,31.6546,-106.3033,
16,Atuona,PF,79.74,few clouds,-9.8000,-139.0333,
17,Butaritari,KI,81.59,scattered clouds,3.0707,172.7902,
18,Noumea,NC,82.99,scattered clouds,-22.2763,166.4572,
22,Rikitea,PF,78.48,clear sky,-23.1203,-134.9692,
23,Malakal,SS,84.07,clear sky,9.5334,31.6605,
25,Hithadhoo,MV,82.99,scattered clouds,-0.6000,73.0833,


In [20]:
# 8a. Create the output File (CSV)
output = ("WeatherPy_vacation.csv")
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output, index_label="City_Name")

In [25]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [27]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))